# Importation, partition train-test et DISQUAL

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

In [2]:
#importation des données
import pandas
df = pandas.read_excel("./data/mushroom.xls")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   capshape           8124 non-null   object
 1   capsurface         8124 non-null   object
 2   capcolor           8124 non-null   object
 3   bruises            8124 non-null   object
 4   gillattachment     8124 non-null   object
 5   gillspacing        8124 non-null   object
 6   gillsize           8124 non-null   object
 7   gillcolor          8124 non-null   object
 8   stalkshape         8124 non-null   object
 9   stalksurfaceabove  8124 non-null   object
 10  stalksurfacebelow  8124 non-null   object
 11  stalkcoloraboveri  8124 non-null   object
 12  stalkcolorbelowri  8124 non-null   object
 13  veilcolor          8124 non-null   object
 14  ringnumber         8124 non-null   object
 15  ringtype           8124 non-null   object
 16  sporeprintcolor    8124 non-null   object


In [3]:
#description
df.describe(include='object')

,capshape,capsurface,capcolor,bruises,gillattachment,gillspacing,gillsize,gillcolor,stalkshape,stalksurfaceabove,...,stalkcoloraboveri,stalkcolorbelowri,veilcolor,ringnumber,ringtype,sporeprintcolor,population,habitat,classe,SAMPLE_STATUS
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,6,4,10,2,2,2,2,12,2,4,...,9,9,4,3,5,9,6,7,2,2
top,x,y,n,f,f,c,b,b,t,s,...,w,w,w,o,p,w,v,d,edible,train
freq,3656,3244,2284,4748,7914,6812,5612,1728,4608,5176,...,4464,4384,7924,7488,3968,2388,4040,3148,4208,4124


In [4]:
#valeurs par variable
df.describe(include='object').loc['unique']

capshape              6
capsurface            4
capcolor             10
bruises               2
gillattachment        2
gillspacing           2
gillsize              2
gillcolor            12
stalkshape            2
stalksurfaceabove     4
stalksurfacebelow     4
stalkcoloraboveri     9
stalkcolorbelowri     9
veilcolor             4
ringnumber            3
ringtype              5
sporeprintcolor       9
population            6
habitat               7
classe                2
SAMPLE_STATUS         2
Name: unique, dtype: object

In [5]:
#partition sur la base de la colonne SAMPLE_STATUS
dfTrain = df.loc[df.SAMPLE_STATUS == 'train'].drop('SAMPLE_STATUS',axis='columns')
dfTest = df.loc[df.SAMPLE_STATUS == 'test'].drop('SAMPLE_STATUS',axis='columns')

#vérif.
print(dfTrain.shape)
print(dfTest.shape)

(4124, 20)
(4000, 20)


In [6]:
dfTrain.head(6)

,capshape,capsurface,capcolor,bruises,gillattachment,gillspacing,gillsize,gillcolor,stalkshape,stalksurfaceabove,stalksurfacebelow,stalkcoloraboveri,stalkcolorbelowri,veilcolor,ringnumber,ringtype,sporeprintcolor,population,habitat,classe
0,f,s,g,f,f,w,b,k,t,f,s,w,w,w,o,e,n,a,g,edible
1,f,y,n,t,f,c,b,n,t,s,s,w,w,w,o,p,n,v,d,edible
2,x,f,g,f,f,w,b,h,t,s,f,w,w,w,o,e,n,s,g,edible
3,f,s,n,f,a,c,b,n,e,s,s,o,o,n,o,p,b,c,l,edible
4,f,f,y,f,f,c,b,g,e,k,k,p,n,w,o,l,h,v,d,poisonous
5,f,y,n,f,f,c,n,b,t,s,k,w,w,w,o,e,w,v,d,poisonous


In [7]:
#méthode DISQUAL instanciation
from discrimintools import DISQUAL
disqual = DISQUAL(n_components=2,target=['classe'],priors="prop",parallelize=False)

In [8]:
# Entrainement
disqual.fit(dfTrain)

DISQUAL(n_components=2, priors='prop', target=['classe'])

In [9]:
disqual.transform(dfTest)

,Z1,Z2
4124,0.243385,-0.461286
4125,-0.014605,0.395672
4126,0.117779,-0.356121
4127,-0.154580,0.356261
4128,0.238605,-0.587712
...,...,...
8119,0.152047,0.211099
8120,0.018136,-0.277977
8121,0.076630,-0.166914
8122,0.237519,0.134281


In [10]:
disqual.predict(X=dfTest)

4124       edible
4125    poisonous
4126       edible
4127    poisonous
4128       edible
          ...    
8119    poisonous
8120       edible
8121       edible
8122       edible
8123       edible
Name: prediction, Length: 4000, dtype: object

In [11]:
disqual.pred_table()

prediction,edible,poisonous
classe,,
edible,2112,48
poisonous,512,1452


In [12]:
disqual.statistics_["chi2"]

,statistic,ddl,pvalue
sporeprintcolor,2296.787306,8,0.000000e+00
ringtype,1512.241854,4,0.000000e+00
gillcolor,1925.736099,11,0.000000e+00
stalksurfaceabove,1435.523703,3,5.764160e-311
stalksurfacebelow,1361.419709,3,6.929806e-295
gillsize,1239.017930,1,2.021746e-271
stalkcoloraboveri,1156.805138,8,2.059443e-244
stalkcolorbelowri,1123.920542,8,2.612410e-237
bruises,1068.596759,1,2.209479e-234
population,1011.036064,5,2.452251e-216
